In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19560604/tmp8jtwiagi
  JVM stdout: /state/partition1/job-19560604/tmp8jtwiagi/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19560604/tmp8jtwiagi/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_mk7516_fi865h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.093 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(223)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(712, 10)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |██████████
03:47:31.30: GLM_1_AutoML_1_20220514_33907 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: 9

█████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  XRT_1_AutoML_1_20220514_33907


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,9950.0,1527442.0,1.0,13.0,4.758894,2.0,27.0,7.463618




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.9142860709765741
RMSE: 0.9561830739856119
LogLoss: 8.027672290288068
Mean Per-Class Error: 0.9451982132886655
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,6,7,8,9,10,...,195,196,197,198,199,200,201,202,Error,Rate
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3 / 3
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4 / 4
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,3 / 3
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,4 / 4
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,2 / 2
9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1 / 1



See the whole table with table.as_data_frame()

Top-10 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.050576,0.018251,0.055944,0.034965,0.070423,0.063380,0.028169
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.949424,0.018251,0.944056,0.965035,0.929577,0.936620,0.971831
3,err_count,135.200000,2.774887,135.000000,138.000000,132.000000,133.000000,138.000000
4,logloss,9.705439,0.872556,9.959799,10.406043,8.748470,8.819287,10.593592
5,max_per_class_error,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,mean_per_class_accuracy,0.525963,0.024430,0.498325,0.538945,0.561139,0.517588,0.513819
7,mean_per_class_error,0.474037,0.024430,0.501675,0.461055,0.438861,0.482412,0.486181
8,mse,0.945549,0.010320,0.941961,0.952740,0.930303,0.945839,0.956904
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-14 04:08:06,24.312 sec,0.0,NaN,NaN,NaN,NaN,NaN
1,,2022-05-14 04:08:07,24.584 sec,5.0,0.964318,21.801947,0.939908,NaN,NaN
2,,2022-05-14 04:08:07,24.850 sec,10.0,0.961017,16.865705,0.928571,NaN,NaN
3,,2022-05-14 04:08:08,25.783 sec,15.0,0.958090,14.587074,0.925457,NaN,NaN
4,,2022-05-14 04:08:08,26.035 sec,20.0,0.958344,12.747243,0.933989,NaN,NaN
5,,2022-05-14 04:08:08,26.303 sec,25.0,0.957023,11.549379,0.929775,NaN,NaN
6,,2022-05-14 04:08:09,27.242 sec,30.0,0.957427,10.535112,0.929775,NaN,NaN
7,,2022-05-14 04:08:10,27.496 sec,35.0,0.955888,9.490823,0.917135,NaN,NaN
8,,2022-05-14 04:08:10,27.760 sec,40.0,0.956289,8.829601,0.919944,NaN,NaN
9,,2022-05-14 04:08:11,28.718 sec,45.0,0.956716,8.485558,0.924157,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,company1,3167.658447,1.000000,0.145381
1,company2,2523.543945,0.796659,0.115819
2,company9,2434.413574,0.768521,0.111728
3,company4,2421.995361,0.764601,0.111158
4,company7,2373.477295,0.749284,0.108932
5,company5,2311.185791,0.729620,0.106073
6,company8,2277.465820,0.718975,0.104525
7,company3,2183.381836,0.689273,0.100207
8,company6,2095.552002,0.661546,0.096176


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XRT_1_AutoML_1_20220514_33907,0.964361,9.70678,0.972396,0.945554
DRF_1_AutoML_1_20220514_33907,0.968555,9.96311,0.973124,0.946971
GBM_grid_1_AutoML_1_20220514_33907_model_8,0.968946,5.61213,0.978418,0.957303
GBM_grid_1_AutoML_1_20220514_33907_model_1,0.972248,5.55617,0.974657,0.949956
GBM_grid_1_AutoML_1_20220514_33907_model_6,0.9742,5.07541,0.980188,0.960768
GBM_2_AutoML_1_20220514_33907,0.976597,20.5252,0.981629,0.963595
GBM_grid_1_AutoML_1_20220514_33907_model_7,0.977006,4.97623,0.979091,0.958619
GBM_grid_1_AutoML_1_20220514_33907_model_5,0.977411,5.17293,0.980211,0.960814
GBM_grid_1_AutoML_1_20220514_33907_model_9,0.977584,4.98982,0.983105,0.966496
GBM_grid_1_AutoML_1_20220514_33907_model_12,0.97762,20.4833,0.981508,0.963359
